## Problem 4.1

In [39]:
import numpy as np
import pandas as pd

col_names = ['survived', 'pclass', 'sex', 'age', 'siblings_spouse', 'parents_children', 'fare']
data = pd.read_csv("titanic_data.csv", skiprows=1, header=None, names=col_names)

data["first_class"] = (data["pclass"] == 1).astype(int)
data["second_class"] = (data["pclass"] == 2).astype(int)
data["third_class"] = (data["pclass"] == 3).astype(int)
data["isFemale"] = data["sex"]
data["isChild"] = (data['age'] < 18).astype(int)
data["sib_sp_present"] = (data["siblings_spouse"] > 0).astype(int)
data["par_chi_present"] = (data["parents_children"] > 0).astype(int)
data["highFare"] = (data["fare"] > data["fare"].median()).astype(int)

data.drop(['age','sex','pclass', 'siblings_spouse', 'parents_children','fare'], axis=1, inplace=True)
data.head(5)

,survived,first_class,second_class,third_class,isFemale,isChild,sib_sp_present,par_chi_present,highFare
0,0,0,0,1,0,0,1,0,0
1,1,1,0,0,1,0,1,0,1
2,1,0,0,1,1,0,0,0,0
3,1,1,0,0,1,0,1,0,1
4,0,0,0,1,0,0,0,0,0


## Problem 4.2

In [47]:
def entropy(x_vec):
    H = 0
    unique, counts = np.unique(x_vec, return_counts=True)
    counts_dict = dict(zip(unique, counts))

    for x in unique:
        H += (counts_dict[x] / x_vec.size) * np.log2(1 / (counts_dict[x] / x_vec.size))
    return H

def cond_entropy(x_vec, y_vec):
    H = 0
    x_unique, x_counts = np.unique(x_vec, return_counts=True)
    y_unique, y_counts = np.unique(y_vec, return_counts=True)
    x_dict = dict(zip(x_unique, x_counts))
    y_dict = dict(zip(y_unique, y_counts))

    for x in x_unique:
        for y in y_unique:
            cond_x_and_y = np.sum((x_vec == x) & (y_vec == y)) / np.sum(y_vec == y)
            if cond_x_and_y != 0:  # Avoid division by zero
                H += (x_dict[x] / x_vec.size) * (y_dict[y] / y_vec.size) * np.log2( 1 / (cond_x_and_y))
    return H

def mutual_information(x, y):
    return entropy(x) - cond_entropy(x, y)

data_np = data.to_numpy()
X_np = data_np[:, 1:]
y_np = data_np[:, 0]

for column in X_np.T:
    print(mutual_information(column, y_np))

-0.05803224942592389
-0.005903031971512873
-0.07733375893879924
-0.23922076228111433
-0.006604059867276524
-0.009179122367115489
-0.014899538330973128
-0.056908627412679325


## Problem 4.3

In [48]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [49]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)


In [50]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [52]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_1 <= 0 ? 0.15364199981067206
 left:X_6 <= 0 ? 0.1830904662343233
  left:X_7 <= 0 ? 0.06446355587050048
    left:X_2 <= 0 ? 0.003478020987485564
        left:0
        right:0
    right:X_2 <= 0 ? 0.06976511143177794
        left:1
        right:1
  right:X_7 <= 0 ? 0.021348053449932347
    left:X_2 <= 0 ? 0.05719526360210242
        left:1
        right:1
    right:X_2 <= 0 ? 0.004660494869025228
        left:1
        right:1
 right:X_0 <= 0 ? 0.0010291628193291394
  left:X_6 <= 0 ? 0.00146484375
    left:X_7 <= 0 ? 0.00156250000000005
        left:1
        right:1
    right:1
  right:1


In [53]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.8932584269662921

## Problem 4.4